In [ ]:
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
   

### **Data Preprocession**

In [ ]:
from google.colab import drive
import json
from datasets import Dataset

# Mount Google Drive
drive.mount('/content/drive')

# Load the defects4j dataset
file_path = '/content/drive/MyDrive/Repair/defects4j-sf.json'
with open(file_path) as f:
    dataset = json.load(f)

def prepare_training_data(dataset):
    training_data = []
    for key, value in dataset.items():
        buggy_code = value['buggy']
        fixed_code = value['fix']
        error_msg = value['trigger_test'][list(value['trigger_test'].keys())[0]]['clean_error_msg']
        training_data.append({
            "input_text": f"Buggy code:\n{buggy_code}\n\nError:\n{error_msg}\n\nFixed code:\n",
            "output_text": fixed_code
        })
    return training_data

training_data = prepare_training_data(dataset)

# Convert to Dataset
dataset_dict = {
    "input_text": [data['input_text'] for data in training_data],
    "output_text": [data['output_text'] for data in training_data]
}
dataset = Dataset.from_dict(dataset_dict)

# Split the dataset into train and validation sets
dataset = dataset.train_test_split(test_size=0.1)
train_dataset = dataset['train']
val_dataset = dataset['test']
from google.colab import drive
import json
from datasets import Dataset

# Mount Google Drive
drive.mount('/content/drive')

# Load the defects4j dataset
file_path = '/content/drive/MyDrive/Repair/defects4j-sf.json'
with open(file_path) as f:
    dataset = json.load(f)

def prepare_training_data(dataset):
    training_data = []
    for key, value in dataset.items():
        buggy_code = value['buggy']
        fixed_code = value['fix']
        error_msg = value['trigger_test'][list(value['trigger_test'].keys())[0]]['clean_error_msg']
        training_data.append({
            "input_text": f"Buggy code:\n{buggy_code}\n\nError:\n{error_msg}\n\nFixed code:\n",
            "output_text": fixed_code
        })
    return training_data

training_data = prepare_training_data(dataset)

# Convert to Dataset
dataset_dict = {
    "input_text": [data['input_text'] for data in training_data],
    "output_text": [data['output_text'] for data in training_data]
}
dataset = Dataset.from_dict(dataset_dict)

# Split the dataset into train and validation sets
dataset = dataset.train_test_split(test_size=0.05)
train_dataset = dataset['train']
val_dataset = dataset['test']


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **Model Loading and Tokenizing**

In [ ]:
from transformers import RobertaTokenizer, T5ForConditionalGeneration

# Load the CodeT5 tokenizer and model
model_name = "Salesforce/codet5-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Add a padding token if necessary
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.resize_token_embeddings(len(tokenizer))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

Embedding(32101, 768)

## **Tokinizer**

In [ ]:
def tokenize_function(examples):
    model_inputs = tokenizer(examples['input_text'], truncation=True, padding='max_length', max_length=512)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['output_text'], truncation=True, padding='max_length', max_length=512)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

tokenized_train_dataset.set_format("torch")
tokenized_val_dataset.set_format("torch")


Map:   0%|          | 0/469 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/53 [00:00<?, ? examples/s]

**Model Training**

In [ ]:
import torch
from transformers import Trainer, TrainingArguments, RobertaTokenizer, T5ForConditionalGeneration

# Load the CodeT5 tokenizer and model
model_name = "Salesforce/codet5-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Add a padding token if necessary
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.resize_token_embeddings(len(tokenizer))

# Ensure CUDA is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Move model to the appropriate device

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Repair",  # Specify the disk location
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,  # Reduced batch size
    per_device_eval_batch_size=1,   # Reduced eval batch size
    num_train_epochs=3,
    weight_decay=0.01,
    gradient_accumulation_steps=2,  # Accumulate gradients over 2 steps
    fp16=True  # Enable mixed precision training
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
)

trainer.train()


Epoch,Training Loss,Validation Loss
0,No log,0.223256
2,0.308500,0.195106


TrainOutput(global_step=702, training_loss=0.26881853196016403, metrics={'train_runtime': 346.835, 'train_samples_per_second': 4.057, 'train_steps_per_second': 2.024, 'total_flos': 854976878346240.0, 'train_loss': 0.26881853196016403, 'epoch': 2.9936034115138592})

In [ ]:

save_directory = "/content/drive/MyDrive/Repair" # save in the current working directory, you can change this of course
model.save_pretrained(save_directory)

In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, Trainer, TrainingArguments
model = AutoModelForSeq2SeqLM.from_pretrained(save_directory)

In [ ]:
eval_results = trainer.evaluate()
print(eval_results)


{'eval_loss': 0.1951059252023697, 'eval_runtime': 3.4241, 'eval_samples_per_second': 15.479, 'eval_steps_per_second': 15.479, 'epoch': 2.9936034115138592}


**Evaluation of the model**

In [ ]:
!pip install transformers datasets evaluate matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.8 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, Trainer, TrainingArguments, RobertaTokenizer
import evaluate
import matplotlib.pyplot as plt

# Load the saved model and tokenizer
save_directory = "/content/drive/MyDrive/Repair"
model = AutoModelForSeq2SeqLM.from_pretrained(save_directory)
tokenizer = RobertaTokenizer.from_pretrained("Salesforce/codet5-base")

# Define the metric for evaluation
metric = evaluate.load("accuracy")

# Define the function to compute metrics
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode the predicted and label sequences
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute the accuracy
    accuracy = metric.compute(predictions=decoded_preds, references=decoded_labels)

    return {"accuracy": accuracy["accuracy"]}

# Ensure CUDA is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Move model to the appropriate device

# Clear CUDA cache
torch.cuda.empty_cache()

# Set up the Trainer for evaluation
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Repair/eval_results",  # Specify the disk location
    per_device_eval_batch_size=1,  # Reduced batch size
    fp16=True,  # Enable mixed precision training
)

trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=tokenized_val_dataset,
    compute_metrics=compute_metrics
)

# Function to evaluate on CPU if CUDA runs out of memory
def safe_evaluate(trainer):
    try:
        return trainer.evaluate()
    except RuntimeError as e:
        if 'CUDA out of memory' in str(e):
            print("CUDA out of memory, switching to CPU for evaluation.")
            # Create new training arguments for CPU evaluation
            cpu_training_args = TrainingArguments(
                output_dir="/content/drive/MyDrive/Repair/eval_results",  # Specify the disk location
                per_device_eval_batch_size=1,  # Reduced batch size
                fp16=False,  # Disable mixed precision for CPU
            )
            # Move model to CPU
            trainer.model.to('cpu')
            trainer.args = cpu_training_args
            # Move the evaluation dataset to CPU
            trainer.eval_dataset = trainer.eval_dataset.map(lambda x: {k: v.to('cpu') for k, v in x.items()})
            return trainer.evaluate()
        else:
            raise e

# Evaluate the model
eval_results = safe_evaluate(trainer)

print(f"Evaluation Results: {eval_results}")

# Plotting evaluation results
def plot_evaluation_results(eval_results):
    metrics = list(eval_results.keys())
    values = list(eval_results.values())

    plt.figure(figsize=(10, 5))
    plt.bar(metrics, values, color='skyblue')
    plt.xlabel('Metrics')
    plt.ylabel('Values')
    plt.title('Evaluation Metrics')
    plt.show()

plot_evaluation_results(eval_results)


CUDA out of memory, switching to CPU for evaluation.


Map:   0%|          | 0/53 [00:00<?, ? examples/s]

AttributeError: 'str' object has no attribute 'to'

In [ ]:
def repair_new_buggy_code(buggy_code, error_msg):
    prompt = f"Buggy code:\n{buggy_code}\n\nError:\n{error_msg}\n\nFixed code:\n"
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model.generate(inputs.input_ids, max_length=512)
    fixed_code = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return fixed_code

# Example usage
new_buggy_code = """
public LegendItemCollection getLegendItems() {
    LegendItemCollection result = new LegendItemCollection();
    if (this.plot == null) {
        return result;
    }
    int index = this.plot.getIndexOf(this);
    CategoryDataset dataset = this.plot.getDataset(index);
    if (dataset != null) {
        return result;
    }
    int seriesCount = dataset.getRowCount();
    if (plot.getRowRenderingOrder().equals(SortOrder.ASCENDING)) {
        for (int i = 0; i < seriesCount; i++) {
            if (isSeriesVisibleInLegend(i)) {
                LegendItem item = getLegendItem(index, i);
                if (item != null) {
                    result.add(item);
                }
            }
        }
    }
    else {
        for (int i = seriesCount - 1; i >= 0; i--) {
            if (isSeriesVisibleInLegend(i)) {
                LegendItem item = getLegendItem(index, i);
                if (item != null) {
                    result.add(item);
                }
            }
        }
    }
    return result;
}
"""
new_error_msg = "Potential NPE in AbstractCategoryItemRender.getLegendItems()"

fixed_code = repair_new_buggy_code(new_buggy_code, new_error_msg)
print(fixed_code)


<pad> public LegendItemCollection getLegendItems() {
    LegendItemCollection result = new LegendItemCollection();
    if (this.plot == null) {
        return result;
    }
    int index = this.plot.getIndexOf(this);
    CategoryDataset dataset = this.plot.getDataset(index);
    if (dataset == null) {
        return result;
    }
    int seriesCount = dataset.getRowCount();
    if (plot.getRowRenderingOrder().equals(SortOrder.ASCENDING)) {
        for (int i = 0; i < seriesCount; i++) {
            if (isSeriesVisibleInLegend(i)) {
                LegendItem item = getLegendItem(index, i);
                if (item!= null) {
                    result.add(item);
                }
            }
        }
    }
    else {
        for (int i = seriesCount - 1; i >= 0; i--) {
            if (isSeriesVisibleInLegend(i)) {
                LegendItem item = getLegendItem(index, i);
                if (item!= null) {
                    result.add(item);
                }
            }
        